# Turbulent backgrounds with nonlinear OSN model

We use the model from Kadakia and Emonet, because OSN adaptation is easy to include into it afterwards. 

The olfactory receptors (OR) have Orco co-receptors. Each OR-Orco complex has an active and an inactive state, with affinities $K^*_{i \mu}$, $K_{i \mu}$ respectively for odor $\mu$, receptor index $i$. Given odor concentrations $c_\mu$, the quasi-static OSN firing rate in response is

$$ A_i = \left[1 + \mathrm{exp} \left( \epsilon_i(t) + \ln \left( \frac{1 + \sum_\mu K_{i \mu} c_\mu }{1 + \sum_\mu K^*_{i \mu} c_\mu }  \right) \right) \right]^{-1} $$

where $\epsilon_i(t)$ is the free energy difference between the unbound states in the inactive and active conformations for OR type $i$, in units of $k_\mathrm{B} T$. This free energy difference changes with feedback from OSN activity, with an adaptation time scale of $250$ ms, but here, we will keep $\epsilon_i$ fixed; the point of the model is to have a nonlinear receptor activation with easy inclusion of adaptation later. 

The affinities $K^*_{i \mu}$ and $K_{i \mu}$ are sampled i.i.d. from a power-law with exponent $\alpha = 0.35$ based on Si et al., 2019. We then ensure that $K^* > K$ (no antagonists) and select a few OSNs per odor to be strongly activated (breaking the power law). 

Note that the power law distribution is over a finite range, 

$$ f_K(k) = \alpha k^{-1-\alpha} \,\, \mathrm{if} k \in [k_\mathrm{low}, k_\mathrm{hi}], \, 0 \, \mathrm{else}$$

and $\alpha > 0$ implies its normalization would blow up at zero, hence there needs to be a cutoff $k_\mathrm{low} > 0$. 

The free energy default value is $5.0$ in Kadakia and Emonet's code, without variance between odors based on the parameters coded in the Github repo. 

### Linear regime for small concentrations
If we assume concentrations are small, we can Taylor expand $A_i$ near $c=0$. We find that

$$ A_i \approx \frac{1}{1 + e^{\epsilon_i(t)}} + \frac{e^{\epsilon_i(t)}}{(1 + e^{\epsilon_i(t)})^2} \sum_{\mu} (K_{i \mu}^* - K_{i \mu}) \, c_{\mu}(t) \,\, .  $$

So, with a large free energy difference to suppress the constant term, this expansion basically reduces to our linear mixture model with odor vectors

$$ \mathbf{s}_{\gamma} = \frac{e^{\epsilon_i(t)}}{(1 + e^{\epsilon_i(t)})^2} \left( \mathbf{K}^*_{\gamma} - \mathbf{K}_{\gamma} \right) \propto \mathbf{K}^*_{\gamma} - \mathbf{K}_{\gamma} $$

Hence, we are looking at a nonlinear manifold with underlying linear odor vectors whose elements are drawn from a power-law. 


### Possible simplifications

Possibly, we could use $\mathbf{K}^*$ drawn from our usual exponential distribution and very small $\mathbf{K}_{i \mu} = 0.01$, so this whole model would reduce to a mildly nonlinear version of our usual manifolds? 

Or we could simplify even further to keep things conceptual, and use a mild Michaelis-Menten saturation function for OSNs, with a MM constant that adapts like the free energy difference of Kadakia and Emonet? That would result in

$$ A_i = \frac{\sum_{\gamma} c_{\gamma} s_{i \gamma}}{e^{\epsilon_i(t)} + \sum_{\gamma} c_{\gamma} s_{i \gamma}} $$

Notice that we can recover exactly this form in the regime where $c$ is not too small, $K \ll K^*$, so $\mathbf{K}^*_i \cdot \mathbf{c} \gg 1$ but $\mathbf{K}_i \cdot \mathbf{c} \ll 1$, and we have

$$ A_i \approx \frac{1}{1 + e^{\epsilon_i(t)} / \mathbf{K^*}_i \cdot \mathbf{c}} 
    = \frac{\sum_\mu  \mathbf{K}^*_{i \mu} \cdot \mathbf{c}}{e^{\epsilon_i(t)} + \mathbf{K^*}_i \cdot \mathbf{c}}  $$

which is the Michaelis-Menten for that we wanted with odor vector $\mathbf{K}^*_\mu$ for each odor





## Implementation

Looking at the code from Kadakia and Emonet, 
https://github.com/elifesciences-publications/ORN-WL-gain-control/blob/master/src/four_state_receptor_CS.py
the $K$ parameters are all equal to $1/10^2 = 0.01$ exactly, while the $1/K^*$ power law ranges from $10^{-4}$ to $10^{-3}$, such that $K^*$ ranges from $1000$ to $10\, 000$
Also, they state in the text that $\alpha = 0.35$, but they actually use $0.38$ in the code. Meanwhile, the original paper reports of fit of $\alpha = 0.42$. I will use $0.38$ since it is in between. 

Then, a few OSNs are randomly selected for each odor to have a larger $K^*$? I will first try without that, it is a bit complicated in Kadakia, they look for OSNs which are weak responders to all odors (average response to all odors at standard conc. and energy difference below $0.03$ in the panel of odors, and make them very responsive to one of the odors, by randomly setting their inverse $K^*$ log-uniform between $10^{-4.5}$ and $10^{-3}$. 

## Functions of general interest

In [ ]:
import numpy as np
from scipy import sparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from time import perf_counter
import os, json
import sys
if "../" not in sys.path:
    sys.path.insert(1, "../")

from modelfcts.ibcm import (
    integrate_inhib_ibcm_network_options,
    ibcm_respond_new_odors,
    compute_mbars_cgammas_cbargammas,
)
from modelfcts.ibcm_analytics import (
    fixedpoint_thirdmoment_exact, 
    ibcm_fixedpoint_w_thirdmoment, 
    ibcm_all_largest_eigenvalues
)
from modelfcts.biopca import (
    integrate_inhib_biopca_network_skip,
    build_lambda_matrix,
    biopca_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.average_sub import (
    integrate_inhib_average_sub_skip, 
    average_sub_respond_new_odors
)
from modelfcts.ideal import (
    find_projector, 
    find_parallel_component, 
    ideal_linear_inhibitor, 
    compute_ideal_factor, 
    compute_optimal_matrices
)
from modelfcts.checktools import (
    analyze_pca_learning
)
from modelfcts.backgrounds import (
    sample_ss_conc_powerlaw,
    update_tc_odor,
    generate_odorant
)
from modelfcts.distribs import (
    truncexp1_average,
    powerlaw_cutoff_inverse_transform, 
    power_range_inverse_transform
)
from modelfcts.tagging import (
    project_neural_tag, 
    create_sparse_proj_mat, 
    SparseNDArray, 
)
from utils.statistics import seed_from_gen
from utils.smoothing_function import (
    moving_average, 
    moving_var
)
from simulfcts.plotting import (
    plot_cbars_gamma_series, 
    plot_w_matrix, 
    plot_background_norm_inhibition, 
    plot_background_neurons_inhibition, 
    plot_pca_results, 
    hist_outline
)
from simulfcts.analysis import compute_back_reduction_stats
from utils.metrics import jaccard, l2_norm

## Main new function: background update with OSN model

In [ ]:
def generate_odor_aff_power(n_rec, rgen, k1val=0.01, 
            k2inv_bounds=(1e-4, 1e-3), alpha=0.35, unit_scale=0.05):
    """ Generate vectors eta and kappa^-1 for an odorant, with antagonism parameter rho.
    Following Kadakia and Emonet, eLife, 2019. 
    
    Our unit concentrations are normalized, so we need to normalize affinities
    as well (the values used by Kadakia and Emonet are EC50 units). 
    To stay in a nearly linear regime where OSNs do not saturate
    we use a factor of 0.01. 
    
    Args:
        n_rec (int): number of receptor types, length of vectors
        rgen (np.random.Generator): random generator (numpy >= 1.17)
        k1val (float): value of all K affinities
        k2inv_bounds (tuple of 2 floats): lowest and highest 1/K* possible
        alpha (float): power law has CDF with exponent alpha, 
            PDF with exponent alpha-1
    
    Returns:
        k1vec (np.ndarray): 1d vector of inactive complex binding affinities
        k2vec (np.ndarray): 1d vector of active complex binding affinities
    """
    # Generate inverse K: all 1/K equal to 100
    k1vec = np.full(n_rec, 0.01 * unit_scale)
    
    # and inverse K*: 1/K* follows power law between k2inv_bounds. 
    # First sample power law elements x in [0, 1], f_X(x) = ax^{a-1}
    # Scipy transforms 0 < x < 1 power-law as 1/K = k_lo + k_hi*x 
    # this seems to be how scipy shifts the powerlaw and what is done in Kadakia and Emonet
    #x = rgen.power(a=alpha, size=n_rec)
    #k2inv = x * k2inv_bounds[1] + k2inv_bounds[0]
    # Same result would be obtained from scipy
    #k2inv = powerlaw.rvs(alpha, loc=k2inv_bounds[0], scale=k2inv_bounds[1], 
    #                     size=n_rec, random_state=rgen)
    
    # The above are within [k_lo, k_hi] indeed, but 1/K doesn't itself follow a power-law 
    # like A*(1/K)^{a-1}, so we implement our own proper power-law with a cutoff:
    r = rgen.random(size=n_rec)
    k2inv = power_range_inverse_transform(r, *k2inv_bounds, alpha)
    
    k2vec = unit_scale / k2inv
    return k1vec, k2vec


# To generate several odors and format parameters the right way
def generate_background_aff_power(n_comp, n_rec, rgen, 
        k1val=0.01, k2inv_bounds=(1e-4, 1e-3), alpha=0.38):
    k1mat, k2mat = [], []
    for i in range(n_comp):
        k1, k2 = generate_odor_aff_power(n_dimensions, rgen_meta, 
                    k1val=k1val, k2inv_bounds=k2inv_bounds, alpha=alpha)
        k1mat.append(k1)
        k2mat.append(k2)
    k1mat = np.stack(k1mat, axis=0)
    k2mat = np.stack(k2mat, axis=0)
    kbothmat = np.stack([k1mat, k2mat], axis=0)
    return kbothmat


def inverse_transform_tanhcdf(r, logb, alpha):
    return (10.0**logb * np.arctanh(r))**(-1.0 / alpha)


def generate_odor_true_si2019(n_rec, rgen, k1val=0.01, 
            alpha=0.49, logb=-1.99, unit_scale=1e-3):
    k1vec = np.full(n_rec, k1val * unit_scale)
    
    r = rgen.random(size=n_rec)
    k2vec = inverse_transform_tanhcdf(r, logb, alpha) * unit_scale
    k2vec = np.clip(k2vec, 0.0, 1e3)
    
    return k1vec, k2vec

def combine_odors_adapt(concs, k1mat, k2mat, epsils, fmax=1.0):
    """ Combine odors coming in with concentrations conc and defined
    by active and inactive binding affinities kappa1, kappa2. 
    OSN types have free energy differences epsils. 

    Args:
        concs (np.ndarray): 1d array of odor concentrations, indexed [n_odors]
        kappa1 (np.ndarray): shape [n_odors, n_osns], affinity of active complexes
        kappa2 (np.ndarray):  shape [n_odors, n_osns], affinity of inactive complexes
        epsils (np.ndarray): shape [n_osns], free energy difference of each OSN type
        fmax (float): maximum amplitude, default 1

    Returns:
        activ (np.ndarray): 1d array of ORN activation, indexed [n_receptors]
    """
    # Dot products over odors
    kc1 = concs.dot(k1mat)
    kc2 = concs.dot(k2mat)
    logterm = (1.0 + kc1) / (1.0 + kc2)
    activs = fmax / (1.0 + np.exp(epsils) * logterm)
    return activs

def combine_odors_compet(concs, kmat, epsils, fmax=1.0):
    r""" Combine odors coming in with concentrations conc and defined
    by binding affinities kmat. OSN types have free energy differences epsils. 
    Approximation of the model by Kadakia and Emonet when K^* >> K, reduces
    to a Michaelis-Menten function of $\sum_\mu K^*_{i \mu} c_\mu$
    with potentially adaptable Michaelis-Menten constant $e^{\epsilon_i(t)}$

    Args:
        concs (np.ndarray): 1d array of odor concentrations, indexed [n_odors]
        kmat (np.ndarray):  shape [n_odors, n_osns], affinity of active complexes
        epsils (np.ndarray): shape [n_osns], free energy difference of each OSN type
        fmax (float): maximum amplitude, default 1

    Returns:
        activ (np.ndarray): 1d array of ORN activation, indexed [n_receptors]
    """
    # Dot products over odors
    kc = concs.dot(kmat)
    activs = fmax * kc / (np.exp(epsils) + kc)
    return activs

In [ ]:
# Function to update the background by combining odorant at the current concentrations with the OSN model
def update_powerlaw_times_concs_aff(tc_bk, params_bk, noises, dt):
    """
    Simulate turbulent odors by pulling wait times until the end of a whiff
        or until the next blank, and a concentration of the whiff.
        For each odor, check whether the time left until switch is <= zero;
        if so, pull either
            - another wait time t_w if current c=0, and pull the new c > 0
              (we were in a blank and are starting a whiff)
            - another wait time t_b if current c > 0, and set c = 0
              (we were in a whiff and are starting a blank)
        Otherwise, decrement t by dt and don't change c.

    Args:
        tc_bk (np.ndarray): array of t, c for each odor in the background,
            where t = time left until next change, c = current concentration
            of the odor. Shaped [n_odors, 2]
        params_bk (list): contains the following elements (a lot needed!):
            whiff_tmins (np.ndarray): lower cutoff in the power law
                of whiff durations, for each odor
            whiff_tmaxs (np.ndarray): upper cutoff in the power law
                of whiff durations, for each odor
            blank_tmins (np.ndarray): same as whiff_tmins but for blanks
            blank_tmaxs (np.ndarray): same as whiff_tmaxs but for blanks
            c0s (np.ndarray): c0 concentration scale for each odor
            alphas (np.ndarray): alpha*c0 is the lower cutoff of p_c
            vecs (np.ndarray): 3d array where axis 0 has length 2, 
                the first sub-array giving the K vectors of each odor, 
                and the second, giving K* vectors. 
            epsils (np.ndarray): free energy difference of each OSN type, 
                shaped [n_osn]. 
            fmax (float): maximum OSN activation amplitude (normalization)
        noises (np.ndarray): fresh U(0, 1) samples, shaped [n_odors, 2],
            in case we need to pull a new t and/or c.
        dt (float): time step duration, in simulation units
    """
    # Update one odor's t and c at a time, if necessary
    tc_bk_new = np.zeros(tc_bk.shape)
    for i in range(tc_bk.shape[0]):
        tc_bk_new[i] = update_tc_odor(tc_bk[i], dt, noises[i],
                                *[p[i] for p in params_bk[:-3]])

    # Compute backgound vector (even if it didn't change)
    k1vecs, k2vecs = params_bk[-3]  # k, k*
    epsils = params_bk[-2]
    fmax = params_bk[-1]
    new_bk_vec = combine_odors_adapt(tc_bk_new[:, 1], k1vecs, k2vecs, epsils, fmax=fmax)
    return new_bk_vec, tc_bk_new

## Initialization

### Aesthetic parameters

In [ ]:
#plt.style.use(['dark_background'])
plt.rcParams["figure.figsize"] = (4.5, 3.0)
do_save_plots = False

In [ ]:
models = ["ibcm", "biopca", "avgsub", "ideal", "orthogonal", "none"]
model_nice_names = {
    "ibcm": "IBCM",
    "biopca": "BioPCA",
    "avgsub": "Average",
    "ideal": "Ideal",
    "orthogonal": "Orthogonal",
    "none": "None"
}
model_colors = {
    "ibcm": "xkcd:turquoise",
    "biopca": "xkcd:orangey brown",
    "avgsub": "xkcd:navy blue",
    "ideal": "xkcd:powder blue",
    "orthogonal": "xkcd:pale rose",
    "none": "grey"
}

### Global model parameters

In [ ]:
# Initialize common simulation parameters
n_dimensions = 50  # Half the real number for faster simulations
n_components = 3  # Number of background odors

inhib_rates = [0.0001, 0.00002]  # alpha, beta  [0.00025, 0.00005]

# Simulation duration
duration = 360000.0
deltat = 1.0
n_chunks = 10
skp = 20 * int(1.0 / deltat)

# Common model options
activ_function = "identity"  #"ReLU"

# Choose randomly generated background vectors
rgen_meta = np.random.default_rng(seed=0x220369e90599ffa80a743d99ac942f28)
#rgen_meta = np.random.default_rng(seed=0x207839aa985b66f211ebb049ecc23647)
#rgen_meta = np.random.default_rng(seed=0x8fec8e034ed20f055ae9f5e0321aaee)

# Background process
update_fct = update_powerlaw_times_concs_aff
# Seed for background simulation, to make sure all models are the same
simul_seed = seed_from_gen(rgen_meta)

# Turbulent background parameters: same rates and constants for all odors
back_params = [
    np.asarray([1.0] * n_components),        # whiff_tmins
    np.asarray([500.] * n_components),       # whiff_tmaxs
    np.asarray([1.0] * n_components),        # blank_tmins
    np.asarray([800.0] * n_components),      # blank_tmaxs
    np.asarray([0.6] * n_components),        # c0s
    np.asarray([0.5] * n_components),        # alphas
]

# Background odors: parameters (K, K^*) and epsils
# K array in back_components is ndexed [kappa_or_eta, n_odors, n_dimensions]
#back_components = generate_background_aff_power(n_components, n_dimensions, rgen_meta)
#back_components = [generate_odorant((n_components, n_dimensions), rgen_meta)*0.01, 
#                  generate_odorant((n_components, n_dimensions), rgen_meta)*50.0]
back_components = generate_odor_true_si2019((n_components, n_dimensions), rgen_meta)
epsils_vec = np.full(n_dimensions, 5.0)
back_params.append(back_components)
back_params.append(epsils_vec)

# To keep OSN amplitudes comparable to usual simulations, scale down OSN max. ampli
max_osn_ampli = 3.0 / np.sqrt(n_dimensions)
back_params.append(max_osn_ampli)

# In the small conc. approx, the odor vectors are (K^* - K)/2
s_gamma_vecs = back_components[1] - back_components[0]
s_gamma_vecs = s_gamma_vecs / l2_norm(s_gamma_vecs, axis=1)[:, None]

# Initial values of background process variables (t, c for each variable)
init_concs = sample_ss_conc_powerlaw(*back_params[:-3], size=1, rgen=rgen_meta)
init_times = powerlaw_cutoff_inverse_transform(
                rgen_meta.random(size=n_components), *back_params[2:4])
tc_init = np.stack([init_times, init_concs.squeeze()], axis=1)

# Initial background vector: combine odors with the tc_init concentrations
init_bkvec = combine_odors_adapt(tc_init[:, 1], 
                back_components[0], back_components[1], epsils_vec, fmax=max_osn_ampli)
# nus are first in the list of initial background params
init_back_list = [tc_init, init_bkvec]

### Check OSN activation distribution is correct

In [ ]:
n_samples = n_dimensions * int(1e5)
bunch_values1, bunch_values2 = generate_odor_aff_power(n_samples, rgen_meta)

In [ ]:
fig, ax = plt.subplots()
kd2 = bunch_values2
counts, binseps = np.histogram(np.log10(kd2), bins="doane")
binwidths = np.diff(10**binseps)
# Center of bins on a log scale, but given in linear coordinates
bin_centers_forlog = 10**((binseps[1:] + binseps[:-1])/2)
pdf = counts / binwidths / kd2.size
ax.bar(x=10**binseps[:-1], align="edge", height=pdf, width=binwidths)
ax.set(yscale="log", xscale="log", xlabel=r"OR-Orco active affinity $K^*$", ylabel="Prob. density")
ax.set(yscale="log", xscale="log", xlabel=r"OR-Orco active affinity $K^*$", ylabel="1-CDF")
plt.show()
plt.close()

In [ ]:
odor_vecs = (bunch_values2 - bunch_values1).reshape(-1, n_dimensions)
averages = []
# Average dot product between them?
chunksize = int(2e3)
nchunks = odor_vecs.shape[0] // chunksize
for i in range(nchunks):
    odors_loc = odor_vecs[i*chunksize:(i+1)*chunksize]
    norms_loc = l2_norm(odors_loc, axis=1)
    dots = odors_loc.dot(odors_loc.T) / np.outer(norms_loc, norms_loc)
    dots[np.diag_indices(dots.shape[0])] = np.nan
    averages.append(np.nanmean(dots))  # ignore diagonal
averages = np.asarray(averages)
print("Average cosine similarity:", np.mean(averages))
print("With bootstrap std:", np.std(averages, ddof=1))

In [ ]:
# Check a heatmap of affinity matrices
fig = plt.figure()
gs = fig.add_gridspec(4, 1)
ax = fig.add_subplot(gs[:3])
cax = fig.add_subplot(gs[3:])
im = ax.imshow(np.log10(back_components[1]), origin="upper", aspect="auto")
ax.set_xlabel("OR type")
ax.set_ylabel("Odor")
fig.colorbar(im, cax=cax, orientation="horizontal", label="OR affinity")
fig.tight_layout()
plt.show()
plt.close()

## IBCM habituation
### IBCM simulation

The good question is: what are IBCM neurons going to learn? The input space is $\vec{x}$, but it's not a linear combination of background odors, which are rather defined by $\vec{\eta}$ and $\vec{\kappa}$. So it's unclear even what exactly will be the input process distribution, what components will be learnt, etc. 

In [ ]:
# IBCM model parameters
n_i_ibcm = 12  # Number of inhibitory neurons for IBCM case

# Model rates
learnrate_ibcm = 0.001  #5e-5
tau_avg_ibcm = 1200  # 2000
coupling_eta_ibcm = 0.6/n_i_ibcm
ssat_ibcm = 50.0
k_c2bar_avg = 0.1
decay_relative_ibcm = 0.005
lambd_ibcm = 1.0
ibcm_rates = [
    learnrate_ibcm, 
    tau_avg_ibcm, 
    coupling_eta_ibcm, 
    lambd_ibcm,
    ssat_ibcm, 
    k_c2bar_avg,
    decay_relative_ibcm 
]
ibcm_options = {
    "activ_fct": activ_function, 
    "saturation": "tanh", 
    "variant": "law", 
    "decay": True
}

# Initial synaptic weights: small positive noise
init_synapses_ibcm = 0.3*rgen_meta.standard_normal(size=[n_i_ibcm, n_dimensions])*lambd_ibcm

In [ ]:
# Run the IBCM simulations
# Perform successive shorter runs/restarts for memory efficiency
tser_ibcm = []
nuser_ibcm = []
bkvecser_ibcm = []
mser_ibcm = []
cbarser_ibcm = []
wser_ibcm = []
yser_ibcm = []
thetaser_ibcm = []
if n_chunks > 1:
    seed_spawns = np.random.SeedSequence(simul_seed).spawn(10)
else:
    seed_spawns = [simul_seed]
for i in range(n_chunks):
    tstart = perf_counter()
    if i == 0:
        init_vari = init_synapses_ibcm
        init_back = init_back_list
    else:
        init_vari = [mser_ibcm[i-1][-1], thetaser_ibcm[i-1][-1], wser_ibcm[i-1][-1]]
        init_back = [nuser_ibcm[i-1][-1], bkvecser_ibcm[i-1][-1]]
    sim_results = integrate_inhib_ibcm_network_options(
                init_vari, update_fct, init_back, 
                ibcm_rates, inhib_rates, back_params, duration/n_chunks, 
                deltat, seed=seed_spawns[i], noisetype="uniform",  
                skp=skp, **ibcm_options
    )
    tser_ibcm.append(sim_results[0] + i/n_chunks*duration)
    nuser_ibcm.append(sim_results[1])
    bkvecser_ibcm.append(sim_results[2])
    mser_ibcm.append(sim_results[3]) 
    cbarser_ibcm.append(sim_results[4]) 
    thetaser_ibcm.append(sim_results[5])
    wser_ibcm.append(sim_results[6])
    yser_ibcm.append(sim_results[7])
    tend = perf_counter()
    print("Finished chunk", i, "in {:.2f} s".format(tend - tstart))

# Concatenate
tser_ibcm = np.concatenate(tser_ibcm, axis=0)
nuser_ibcm = np.concatenate(nuser_ibcm)
bkvecser_ibcm = np.concatenate(bkvecser_ibcm)
mser_ibcm = np.concatenate(mser_ibcm)
cbarser_ibcm = np.concatenate(cbarser_ibcm)
thetaser_ibcm = np.concatenate(thetaser_ibcm)
wser_ibcm = np.concatenate(wser_ibcm)
yser_ibcm = np.concatenate(yser_ibcm)

### Background process plot

In [ ]:
if n_dimensions <= 2:
    fig, ax = plt.subplots()
    # Use a different color for 3 cases: either odor absent, both odors
    where_12 = (nuser_ibcm[:, :, 1] > 0).astype(bool)
    where_both = np.logical_and(where_12[:, 0], where_12[:, 1])
    where_1 = np.logical_and(where_12[:, 0], ~where_12[:, 1])
    where_2 = np.logical_and(~where_12[:, 0], where_12[:, 1])
    ax.scatter(bkvecser_ibcm[where_both, 0], bkvecser_ibcm[where_both, 1], label="Both odors")
    ax.scatter(bkvecser_ibcm[where_1, 0], bkvecser_ibcm[where_1, 1], label="Odor 0 only")
    ax.scatter(bkvecser_ibcm[where_2, 0], bkvecser_ibcm[where_2, 1], label="Odor 1 only")
    vecs = np.zeros(s_gamma_vecs.shape)
    scale = 0.5
    orig = np.zeros([3, n_components])
    for i in range(n_components):
        vecs[i] = s_gamma_vecs[i] / np.sqrt(np.sum(s_gamma_vecs[i]**2)) * scale
        ax.annotate("", xytext=(0, 0), xy=s_gamma_vecs[i], 
                    arrowprops=dict(width=2.0, color="k"))
    figname = "background_two_odors_2d_affinity_model.pdf"
    zlbl = None
elif n_components >= 3:
    dims = (1, 2, 4)
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    fig.set_size_inches(6.0, 3.0)
    where_123 = (nuser_ibcm[:, :, 1] > 0).astype(bool)
    locations = {
        "All odors": np.all(where_123, axis=1),  #all
        "Odors 0&1": (where_123[:, 0] & where_123[:, 1] & ~where_123[:, 2]),  # 12
        "Odors 0&2": (where_123[:, 0] & ~where_123[:, 1] & where_123[:, 2]),  # 13
        "Odors 1&2": (~where_123[:, 0] & where_123[:, 1] & where_123[:, 2]),  # 23
        "Odor 0": (where_123[:, 0] & ~where_123[:, 1] & ~where_123[:, 2]),  #1
        "Odor 1": (~where_123[:, 0] & where_123[:, 1] & ~where_123[:, 2]), 
        "Odor 2": (~where_123[:, 0] & ~where_123[:, 1] & where_123[:, 2])
    }
    all_colors = {
        "All odors": "xkcd:grey",
        "Odors 0&1": "xkcd:orange",
        "Odors 0&2": "xkcd:purple",
        "Odors 1&2": "xkcd:green",
        "Odor 0": "xkcd:red",
        "Odor 1": "xkcd:yellow",
        "Odor 2": "xkcd:blue"
        
    }
    for lbl, slc in locations.items():
        ax.scatter(bkvecser_ibcm[slc, dims[0]], bkvecser_ibcm[slc, dims[1]], 
                   bkvecser_ibcm[slc, dims[2]], label=lbl, color=all_colors[lbl])
    vecs = np.zeros(s_gamma_vecs.shape)
    scale = 1.0
    orig = np.zeros([3, n_components])
    for i in range(n_components):
        vecs[i] = s_gamma_vecs[i] / np.sqrt(np.sum(s_gamma_vecs[i]**2)) * scale
    ax.quiver(*orig, *(vecs[:, dims].T), color="k", lw=2.0, arrow_length_ratio=0.2)
    ax.scatter(0, 0, 0, color="k", s=100)
    ax.view_init(azim=120, elev=35)
    figname = "background_manifold_three_odors_affinity_model.pdf"
    ax.set(xlabel="OSN {}".format(dims[0]), ylabel="OSN {}".format(dims[1]))
    zlbl = ax.set_zlabel("OSN {}".format(dims[2]))
elif n_components == 2:
    dims = (0, 1, 4)
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    fig.set_size_inches(6.0, 3.0)
    where_12 = (nuser_ibcm[:, :, 1] > 0).astype(bool)
    locations = {
        "Odor 0": (where_12[:, 0] & ~where_12[:, 1]),  # 1
        "Odor 1": (~where_12[:, 0] & where_12[:, 1]),  # 2
        "Both odors": np.all(where_12, axis=1)  # all
    }
    all_colors = {
        "Odor 0": "tab:blue",
        "Odor 1": "tab:orange",
        "Both odors": "tab:green",
        
    }
    for i, lbl in enumerate(locations.keys()):
        slc = locations[lbl]
        ax.scatter(bkvecser_ibcm[slc, dims[0]], bkvecser_ibcm[slc, dims[1]], 
                   bkvecser_ibcm[slc, dims[2]], label=lbl, color=all_colors[lbl], zorder=3-i)
    vecs = np.zeros(s_gamma_vecs.shape)
    scale = 0.5
    orig = np.zeros([3, n_components])
    for i in range(n_components):
        vecs[i] = s_gamma_vecs[i] / np.sqrt(np.sum(s_gamma_vecs[i]**2)) * scale
    ax.quiver(*orig, *(s_gamma_vecs[:, dims].T), color="k", lw=2.0)
    ax.scatter(0, 0, 0, color="k", s=100)
    ax.view_init(elev=22)
    figname = "background_two_odors_3d_affinity_model.pdf"
    ax.set(xlabel="OSN {}".format(dims[0]), ylabel="OSN {}".format(dims[1]))
    zlbl = ax.set_zlabel("OSN {}".format(dims[2]))
leg = ax.legend(loc="upper right", bbox_to_anchor=(0.0, 1.0))
fig.tight_layout()
if do_save_plots:
    fig.savefig(pj("..", "figures", "adaptation", figname), 
                transparent=True, bbox_inches="tight", bbox_extra_artists=(zlbl, leg))
plt.show()
plt.close()

In [ ]:
# Background vectors time series with mixed concentrations
tslice = slice(0, 50000, 200)
n_cols = 6
n_plots = 24 // 2  # Only show first 24 OSNs
n_rows = n_plots // n_cols + min(1, n_plots % n_cols)
fig, axes = plt.subplots(n_rows, n_cols, sharex=True, sharey=True)
fig.set_size_inches(n_cols*1.25, n_rows*1.25)
for i in range(n_plots):
    ax = axes.flat[i]
    ax.scatter(bkvecser_ibcm[tslice, 2*i+1], bkvecser_ibcm[tslice, 2*i], 
               s=9, alpha=0.5, color="k")
    for j in range(n_components):
        ax.plot(*zip([0.0, 0.0], s_gamma_vecs[j, 2*i:2*i+2:][::-1]), lw=2.0)
    ax.set(xlabel="OSN {}".format(2*i+2), ylabel="OSN {}".format(2*i+1))
for i in range(n_plots, n_rows*n_cols):
    axes.flat[i].set_axis_off()
fig.tight_layout()
#if do_save_plots:
#    fig.savefig(pj("..", "figures", "correlation", "osn_background_vectors.pdf"), 
#               transparent=True, bbox_inches="tight")
plt.show()
plt.close()

### Plotting the time course of the different neurons

In [ ]:
# Calculate cgammas_bar and mbars
transient = int(5/6*duration / deltat) // skp
# Dot products \bar{c}_{\gamma} = \bar{\vec{m}} \cdot \vec{x}_{\gamma}
mbarser, c_gammas, cbars_gamma = compute_mbars_cgammas_cbargammas(
                                    mser_ibcm, coupling_eta_ibcm, s_gamma_vecs)
sums_cbars_gamma = np.sum(cbars_gamma, axis=2)
sums_cbars_gamma2 = np.sum(cbars_gamma*cbars_gamma, axis=2)

# Analytical prediction, exact: need moments of nu. Easiest to compute numerically. 
conc_ser = nuser_ibcm[:, :, 1]
# Odors are all iid so we can average over all odors
mean_conc = np.mean(conc_ser)
sigma2_conc = np.var(conc_ser)
thirdmom_conc = np.mean((conc_ser - mean_conc)**3)
moments_conc = [mean_conc, sigma2_conc, thirdmom_conc]

# Analytical prediction
res = fixedpoint_thirdmoment_exact(moments_conc, 1, n_components-1)
c_specif, c_nonspecif = res[:2]
cs_cn = res[:2]

# Count how many dot products are at each possible value. Use cbar = 1.0 as a split. 
split_val = 2.0
cbars_gamma_mean = np.mean(cbars_gamma[transient:], axis=0)
cgammas_bar_counts = {"above": int(np.sum(cbars_gamma_mean.flatten() > split_val)), 
                      "below": int(np.sum(cbars_gamma_mean.flatten() <= split_val))}
print(cgammas_bar_counts)

specif_gammas = np.argmax(np.mean(cbars_gamma[transient:], axis=0), axis=1)
print(specif_gammas)

# Analytical W
analytical_w = ibcm_fixedpoint_w_thirdmoment(inhib_rates, moments_conc, s_gamma_vecs, cs_cn, specif_gammas)

### IBCM habituation analysis

In [ ]:
fig, ax = plt.subplots()
#ax.plot(tser_ibcm[:300], nuser_ibcm[:300, :, 1])
neurons_cmap = sns.color_palette("Greys", n_colors=n_i_ibcm)
for i in range(n_i_ibcm):
    ax.plot(tser_ibcm/1000, thetaser_ibcm[:, i], lw=0.5, color=neurons_cmap[i])
ax.set(xlabel="Time (x1000 steps)", ylabel=r"$\bar{\Theta} = \bar{c}^2$ moving average")
plt.show()
plt.close()

In [ ]:
fig , ax, _ = plot_cbars_gamma_series(tser_ibcm, cbars_gamma, 
                        skp=10, transient=320000 // skp)
# Compare to exact analytical fixed point solution
#ax.set_xlim([350, 360])
ax.axhline(c_specif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{specific}}$")
ax.axhline(c_nonspecif, ls="--", color="grey", 
           label=r"Analytical $\bar{c}_{\gamma=\mathrm{non}}$")
fig.tight_layout()
leg = ax.legend(loc="upper left", bbox_to_anchor=(1., 1.))

#fig.savefig("figures/powerlaw/cbargammas_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight", bbox_extra_artists=(leg,))
plt.show()
plt.close()

In [ ]:
# Correlation between nu's and c's, see if some neurons are specific to odors
# Each neuron turns out to correlate its response to  one concentration
# that means it is specific to that odor. 
cbarser_norm_centered = cbarser_ibcm - np.mean(cbarser_ibcm[transient:], axis=0)
conc_ser_centered = (nuser_ibcm[:, :, 1] 
                     - np.mean(nuser_ibcm[transient:, :, 1], axis=0))
correl_c_nu = np.mean(cbarser_norm_centered[transient:, :, None] 
                      * conc_ser_centered[transient:, None, :], axis=0)

fig, ax = plt.subplots()
img = ax.imshow(correl_c_nu.T)
ax.set(ylabel=r"Component $\gamma$", xlabel=r"Neuron $i$")
fig.colorbar(img, label=r"$\langle (\bar{c}^i - \langle \bar{c}^i \rangle)"
             r"(\nu_{\gamma} - \langle \nu_{\gamma} \rangle) \rangle$", 
            location="top")
fig.tight_layout()
#fig.savefig("figures/powerlaw/specificities_turbulent_background_example.pdf", 
#           transparent=True, bbox_inches="tight")
plt.show()
plt.close()

# Check if each component has at least one neuron
for comp in range(n_components):
    print("Number of neurons specific to component {}: {}".format(
            comp, np.sum(np.mean(cbars_gamma[-2000:, :, comp], axis=0) > split_val)))

In [ ]:
fig, axes, _ = plot_background_neurons_inhibition(tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=1)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_ibcm, bkvecser_ibcm, yser_ibcm, skp=1)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
#fig.savefig("figures/powerlaw/pn_activity_norm_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

In [ ]:
fig, axes = plot_w_matrix(tser_ibcm, wser_ibcm, skp=100)
fig.tight_layout()
#fig.savefig("figures/powerlaw/w_series_turbulent_background_example.pdf", 
#            transparent=True, bbox_inches="tight")
plt.show()
plt.close()

### Stability of average fixed points
Check the eigenvalues of the jacobian for one neuron, for every possible specificity. There are $2^{n_B}$ possibilities: choosing specific or not for each odor

That calculation does not really work here, since the background process is not at all the linear superposition used analytically. 

In [ ]:
all_max_eigenvalues = ibcm_all_largest_eigenvalues(
    moments_conc, ibcm_rates, s_gamma_vecs, m3=1.0, cut=1e-16, options=ibcm_options
)

In [ ]:
fig, ax = plt.subplots()
ibcm_specif_keys = list(all_max_eigenvalues.keys())
ibcm_eig_values = np.asarray([all_max_eigenvalues[a] for a in ibcm_specif_keys])
reals, imags = np.real(ibcm_eig_values), np.imag(ibcm_eig_values)
ibcm_eig_values_specif1 = np.asarray([len(s) == 1 for s in ibcm_specif_keys], dtype=bool)
highlights = ibcm_eig_values_specif1
ax.axvline(0.0, ls="--", color="k", lw=1.0)
ax.axhline(0.0, ls="--", color="k", lw=1.0)
scaleup = 1e3
ax.plot(reals[highlights]*scaleup, imags[highlights]*scaleup, marker="*", mfc="b", mec="b", 
        ls="none", label="One odor", ms=8)
ax.plot(reals[~highlights]*scaleup, imags[~highlights]*scaleup, marker="o", mfc="k", mec="k", 
       ls="none", label="0 or 2+ odors", ms=6)
for side in ("top", "right"):
    ax.spines[side].set_visible(False)
ax.legend(title="Specificity")
ax.set(xlabel=r"$\mathrm{Re}(\lambda_{\mathrm{max}})$    ($\times 10^{-3}$)", 
      ylabel=r"$\mathrm{Im}(\lambda_{\mathrm{max}})$     ($\times 10^{-3}$)")
fig.tight_layout()
plt.show()
plt.close()

## BioPCA simulation

### BioPCA habituation simulation

In [ ]:
# BioPCA model parameters
n_i_pca = n_components * 2  # Number of inhibitory neurons for BioPCA case

# Model rates
learnrate_pca = 3e-5  # Learning rate of M
# Choose Lambda diagonal matrix as advised in Minden et al., 2018
# but scale it up to counteract W regularization
lambda_range_pca = 0.5
lambda_max_pca = 8.0
# Learning rate of L, relative to learnrate. Adjusted to Lambda in the integration function
rel_lrate_pca = 2.0  #  / lambda_max_pca**2 
lambda_mat_diag = build_lambda_matrix(lambda_max_pca, lambda_range_pca, n_i_pca)

xavg_rate_pca = learnrate_pca
pca_options = {
    "activ_fct": activ_function, 
    "remove_lambda": False, 
    "remove_mean": True
}
biopca_rates = [learnrate_pca, rel_lrate_pca, lambda_max_pca, lambda_range_pca, xavg_rate_pca]


# Initial synaptic weights: small positive noise
# We selected a seed (out of 40+ tested) giving initial conditions leading to correct PCA
# The model has trouble converging on this background, we're giving as many chances as possible here. 
rgen_pca = np.random.default_rng(seed=0xa135db8c88ef9466dcf2320262a588be)
init_synapses_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_i_pca)
init_mmat_pca = rgen_pca.standard_normal(size=[n_i_pca, n_dimensions]) / np.sqrt(n_dimensions)
init_lmat_pca = np.eye(n_i_pca, n_i_pca)  # Supposed to be near-identity, start as identity
ml_inits_pca = [init_mmat_pca, init_lmat_pca]

In [ ]:
# Run simulation
sim_results = integrate_inhib_biopca_network_skip(
                ml_inits_pca, update_fct, init_back_list, biopca_rates, 
                inhib_rates, back_params, duration, deltat, 
                seed=simul_seed, noisetype="uniform", skp=skp, **pca_options)
(tser_pca, 
 nuser_pca, 
 bkvecser_pca, 
 mser_pca, 
 lser_pca, 
 xser_pca, 
 cbarser_pca, 
 wser_pca, 
 yser_pca) = sim_results

### BioPCA simulation analysis

In [ ]:
res = analyze_pca_learning(bkvecser_pca, mser_pca, lser_pca, 
                           lambda_mat_diag, demean=pca_options["remove_mean"])
true_pca, learnt_pca, fser, off_diag_l_avg_abs, align_error_ser = res

In [ ]:
from utils.statistics import principal_component_analysis
from modelfcts.checktools import compute_pca_meankept

In [ ]:
fig, axes = plot_pca_results(tser_pca/1000, true_pca, learnt_pca, align_error_ser, off_diag_l_avg_abs)
axes[-1].set_xlabel("Time (x1000 steps)")
fig.set_size_inches(fig.get_size_inches()[0], 3*2.5)
plt.show()
plt.close()

In [ ]:
fig, ax, bknorm_ser, ynorm_ser = plot_background_norm_inhibition(
                                tser_pca, bkvecser_pca, yser_pca, skp=10)

# Compute noise reduction factor, annotate
transient = 100000 // skp
norm_stats = compute_back_reduction_stats(bknorm_ser, ynorm_ser, trans=transient)

print("Mean activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['avg_reduction'] * 100))
print("Standard deviation of activity norm reduced to "
      + "{:.1f} % of input".format(norm_stats['std_reduction'] * 100))
ax.annotate("St. dev. reduced to {:.1f} %".format(norm_stats['std_reduction'] * 100), 
           xy=(0.98, 0.98), xycoords="axes fraction", ha="right", va="top")

ax.legend(loc="center right", bbox_to_anchor=(1.0, 0.8))
fig.tight_layout()
plt.show()
plt.close()